Upload this notebook to CoLab and run the cells. 

### Prepare

* Download code, dataset and test zip file from public [google drive link](https://drive.google.com/file/d/1HdQBnxkeeHSGITX3m2Qa12cZr5iIo1Ud/view?usp=sharing) (since github repo is private)
* Download [weights file here](https://drive.google.com/file/d/1qjAJwsS6Rx2niTIVbak24oKlLitPWZU-/view?usp=sharing)
* Upload all the downloaded files back to the colab notebook. 

Run the following cell to unzip the files, move them and install the required python libraries.

In [ ]:
%%sh

sudo apt-get install unzip
unzip -q code_dataset_test.zip
mv code_dataset_test/* .
rm code_dataset_test.zip
pip install -r requirements.txt

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Using Resnet-101 backbone pretrained on COCO, train only the newly added heads 

In [ ]:
!python train.py --dataset dataset/ --weights "coco" --training_type "heads" --epochs 29

## Fine tune the network

In [ ]:
!python train.py --dataset dataset/ --weights "last" --training_type "all" --learning_rate 1e-6 --epochs 30

## Evaulation

Here you can use the weights which is trained above. `mask_rcnn_facemask_0030.h5` is the weight file downloaded from Google Drive and uploaded back to CoLab.

In [ ]:
!python eval.py --dataset dataset/ --weights "weights/mask_rcnn_facemask_0030.h5" --device "gpu"

## Visualizing masks in unseen images

In [4]:
weights_path = 'weights/mask_rcnn_facemask_0030.h5'
test_image_dir = 'test'

In [ ]:
from mrcnn.model import MaskRCNN
from facemask import FaceMaskConfig

class InferenceConfig(FaceMaskConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

cfg = InferenceConfig()

mdl = MaskRCNN(mode="inference", model_dir='logs',
               config=cfg)
mdl.load_weights(weights_path, by_name=True)

In [ ]:
from infer import visualize_masks

visualize_masks(test_image_dir, mdl)